<a href="https://colab.research.google.com/github/TAKE-JP-17/Pytorch/blob/main/ODEode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf

# Define the NN
class PINN(tf.keras.Model):
  super(PINN, self).__init__()
  self.dense1 = tf.keras.layers.Dense(64, activation='tanh')
  self.dense2 = tf.keras.layers.Dense(64, activation='tanh')
  self.dense3 = tf.keras.layers.Dense(2, activation=None) # Output layer: [x(t), y(t)]

  def call(self, t):
    t = self.dense1(t)
    t = self.dense2(t)
    return = self.dense3(t)

# Define the Loss Function
def loss_fn(model, t):
    with tf.GradientTape() as tape1, tf.GradientTape() as tape2:
        tape1.watch(t)
        tape2.watch(t)
        output = model(t)
        x = output[:, 0:1]
        y = output[:, 1:2]

        dx_dt = tape1.gradient(x, t)
        dy_dt = tape2.gradient(y, t)

        # Residuals from the ODEs
        res_x = dx_dt - 2 * x + y
        res_y = dy_dt - x + 2 * y

        # Initial condition losses
        init_loss_x = tf.square(x[0] - 1)
        init_loss_y = tf.square(y[0] - 0)

        # Total Loss
        loss = tf.reduce_mean(tf.square(res_x)) + tf.reduce_mean(tf.square(res_y)) + init_loss_x + init_loss_y
    return loss

# Define training function
def train(model, t, epochs, optimizer):
  for epoch in range(epochs):
    with tf.GradientTape() as tape:
      loss = loss_fn(model, t)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    if epoch % 500 == 0:
      print(f'Epoch {epoch}: Loss = {loss.numpy()}')

# Create the model and optimizer
model = PINN()
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

# Training Domain
t = tf.convert_to_tensor(np.linspace(0, 5, 100)[:, None], dtype=tf.float32)

# Train the model
train(model, t, epochs=10000, optimizer=optimizer)




